<a href="https://colab.research.google.com/github/skjdfhkskjds/machine-learning-projects/blob/main/heart_disease_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [70]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [71]:
url = 'https://raw.githubusercontent.com/skjdfhkskjds/machine-learning-projects/main/heart.csv'
df = pd.read_csv(url)
df.tail()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1
917,38,M,NAP,138,175,0,Normal,173,N,0.0,Up,0


In [72]:
df.isnull().sum().sort_values(ascending=False)/df.shape[0]

HeartDisease      0.0
ST_Slope          0.0
Oldpeak           0.0
ExerciseAngina    0.0
MaxHR             0.0
RestingECG        0.0
FastingBS         0.0
Cholesterol       0.0
RestingBP         0.0
ChestPainType     0.0
Sex               0.0
Age               0.0
dtype: float64

In [73]:
features = ['Age','Sex','ChestPainType','RestingBP','Cholesterol','FastingBS','RestingECG','MaxHR','ExerciseAngina','Oldpeak','ST_Slope']
numerical_features = ['Age','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak']
categorical_features = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']

In [74]:
#Instantiate SimpleImputer 
si=SimpleImputer(missing_values = np.nan, strategy="median")
si.fit(df[numerical_features])
  
#Filling missing data with median
df[numerical_features] = si.transform(df[numerical_features])

In [75]:
for category in categorical_features:
  df[category] = pd.factorize(df[category])[0]

#Adds a new feature called type of patient based on the kmeans clustering of the dataset
kmeans = KMeans(n_clusters=2)
kmeans.fit(df[features])
df['typeOfPatient'] = kmeans.predict(df[features])

In [76]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,typeOfPatient
0,40.0,0,0,140.0,289.0,0.0,0,172.0,0,0.0,0,0,0
1,49.0,1,1,160.0,180.0,0.0,0,156.0,0,1.0,1,1,0
2,37.0,0,0,130.0,283.0,0.0,1,98.0,0,0.0,0,0,0
3,48.0,1,2,138.0,214.0,0.0,0,108.0,1,1.5,1,1,0
4,54.0,0,1,150.0,195.0,0.0,0,122.0,0,0.0,0,0,0


In [77]:
#splits the dataset into 80:20 training and testing data
y = df.pop('HeartDisease')
dftrain, dftest, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

In [78]:
from sklearn.preprocessing import StandardScaler
#Scaling numeric features using sklearn StandardScalar
sc=StandardScaler()
dftrain[numerical_features]=sc.fit_transform(dftrain[numerical_features])
dftest[numerical_features]=sc.transform(dftest[numerical_features])

In [79]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation=tf.nn.relu),
  	keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [80]:
model.fit(dftrain, y_train, epochs=60, batch_size=1)

Epoch 1/60
734/734 [==============================] - 2s 2ms/step - loss: 0.4841 - accuracy: 0.7888
Epoch 2/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3872 - accuracy: 0.8420
Epoch 3/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3625 - accuracy: 0.8542
Epoch 4/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3437 - accuracy: 0.8583
Epoch 5/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3313 - accuracy: 0.8692
Epoch 6/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3199 - accuracy: 0.8733
Epoch 7/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3157 - accuracy: 0.8733
Epoch 8/60
734/734 [==============================] - 1s 2ms/step - loss: 0.3066 - accuracy: 0.8801
Epoch 9/60
734/734 [==============================] - 1s 2ms/step - loss: 0.2989 - accuracy: 0.8883
Epoch 10/60
734/734 [==============================] - 1s 2ms/step - loss: 0.2918 - accuracy: 0.8815

In [83]:
test_loss, test_acc = model.evaluate(dftest, y_test)
print('Test accuracy:', test_acc)

6/6 [==============================] - 0s 2ms/step - loss: 0.6370 - accuracy: 0.8533
Test accuracy: 0.85326087474823


In [84]:
model.save('heart_disease_prediction.h5')